# MIDI Concepts and Representations

In [1]:
import sys
import shutil
import os
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt

In [2]:
import gmidi.midiarray as arrlib
import gmidi.pypianoroll as mullib
import gmidi.pretty_midi as prelib
import mido as fillib
import midi as patlib

### MIDI Concepts

<div style="-webkit-column-count: 2; -moz-column-count: 2; column-count: 2; -webkit-column-rule: 1px dotted #e0e0e0; -moz-column-rule: 1px dotted #e0e0e0; column-rule: 1px dotted #e0e0e0;">
    <div style="display: inline-block;">
        <h4>Vars</h4><p>(number of)</p>
        <ul>
          <li>Notes
            <ul>
              <li>wn - whole note</li>
              <li>hn - half note</li>
              <li>qn - quarter note</li>
              <li>en - 8th note</li>
              <li>sn - 16th note</li>
              <li>ttn - 32nd note</li>
            </ul>
          </li>
          <li>Time
            <ul>
              <li>m - minutes</li>
              <li>s - seconds</li>
              <li>ms - microsecond</li>
            </ul>
          </li>
          <li>Music Related
            <ul>
              <li>bar - bar</li>
              <li>b - beat</li>
              <li>c - clock</li>
            </ul>
          </li>
        </ul>
    </div>
    <div style="display: inline-block;">
    </div>
    <div style="display: inline-block;">
        <h4>Basic Relations</h4>
        <ul>
          <li>Notes
            <ul>
              <li>hn = 2 * wn</li>
              <li>qn = 2 * hn</li>
              <li>en = 2 * qn</li>
              <li>sn = 2 * en</li>
              <li>ttn = 2 * sn</li>
            </ul>
          </li>
          <li>Time
            <ul>
              <li>s = 60 * m</li>
              <li>ms = 10^6 s</li>
            </ul>
          </li>
        </ul>
        
    </div>
</div>

#### MIDI Header
_______________________________
It defines:

* The number of tracks 
* The ticks per beat parameter
* The type of file

> **type 0 (single track)**: all messages are saved in one track

> **type 1 (synchronous)**: all tracks start at the same time and have the same tempo and time-signature 

> **type 2 (asynchronous)**: each track is independent of the others

________________________
#### MIDI times

N| Name | Defined at | Form | Common Values |
-|--------------------------------------| --- | --- | --- 
1| Ticks per Beat 				(TPB)	| Header 				| t = TPB * b			| usually TPB = TPQN             
2| 32nd Note per Beat 			(TTNPB)	| Time Signature Event 	| ttn = TTNPB * b 		| usually TTNPB = 8              
3| Beat per Minute 			(BPM)	    | - 					| b = BPM * m 			|                                
4| Tempo 						(tempo)	| Tempo Event			| ms/b =  60 * 10^6/BPM	|                                
5| Ticks per Quarter Note 		(TPQN)	| - 					| t = TPQN * qn 		| usually TPQN = TPB             
6| Time Signature Numerator	(TSN)	    | Time Signature Event 	| bar = (TSD * wn)/TSN	| 1,2,3,4,6,9,12 most common     
7| Time Signature Denominator 	(TSD)	| Time Signature Event 	| bar = (TSD * wn)/TSN	| 2,4,8 most common              
8| Clicks per Metronome Clock	(CPMC)	| Time Signature Event 	| cl = CPMC * c	        | 24 click per beat             

_________________________
#### Tracks, Channels, Programs, Voices and Instruments

There are three types of MIDI files:

* **Track**: a MIDI track is a structure with 16 channels and represents a sequence of MIDI events. The type of MIDI file dictates how these tracks are related to each other. Usually one track is represented by a staff in scores editing softwares.

* **Channels**: each channel may have one and only one programm associated at each tick, and most of the MIDI events such as notes 'on' and 'off' events must be associated with one channel. This way in one track we may play different programs by changing the program of one channel along the file or associating notes to different channels with different programs. 

* **Program**: in standart MIDI there are 128 programs that control the sound generating parameters, but it is possible to have by including banks each one with 128 programs and an event of selecting bank. These programs are identifiers for a set of parameters that control the way a note will sound and are related to the timber.

* **Instrument**: 1) most of the score editing tools have instruments structures corresponding to real musical instruments; these are a set of programs that represent different the parameters for the different timbers that those instruments can sound, for example in *Musescore* when one staff is associated with the `Violin` instrument this automatically sets three different programs to three different channels correponding to *arco*, *pizzicato* and *tremolo* programs; 2) a structure developed in pretty_midi package that corresponds to a triple (track,channel,program) and represent a set of notes

* **Voice**: in most of the scores editing softwares we have the concept of voice, usually 4 per staff, this concept do not map directly in a MIDI representation, but let us achieve a more readable representation of the sheet and let us associate different voices with different channels which allows us to play different programs within the same track at the same time.




____________________________________

## Defining representations

In [3]:
def pretty_save(src,path):
    src.write(path)
    return path

def pretty_load(path,res):
    midi_data = prelib.PrettyMIDI(path,resolution=res)
    return midi_data

pretty_repr = {"load": pretty_load, "save": pretty_save, "instance": prelib.PrettyMIDI()}

In [4]:
def change_res_pat(pattern,n_res):
        o_res = pattern.resolution
        
        if o_res == n_res:
            return pattern
        
        frac = n_res/o_res
        for t in pattern:
            for e in t:
                e.tick = int(round(frac*e.tick))
        pattern.resolution = n_res

        return pattern

def pattern_save(src,path):
    patlib.write_midifile(path, src)
    return path

def pattern_load(path,res):
    midi_data = patlib.read_midifile(path)
    change_res_pat(midi_data,res)
    return midi_data

pattern_repr = {"load": pattern_load, "save": pattern_save, "instance": patlib.Pattern()}


In [5]:
def multitrack_save(src,path):
    src.write(path)
    return path

def multitrack_load(path,res):
    mul = mullib.Multitrack(beat_resolution=res,name=os.path.basename(path))
    pretty = pretty_load(path,res)
    mul.parse_pretty_midi(pretty,skip_empty_tracks=False)       
    return mul

multitrack_repr = {"load": multitrack_load, "save": multitrack_save, "instance": mullib.Multitrack()}


In [6]:
def midiarray_save(src,path):
    src.save(path)
    return path

def midiarray_load(path,res):
    midi_data = arrlib.MidiArray()
    midi_data.load(path,res)
    return midi_data

midiarray_repr = {"load": midiarray_load, "save": midiarray_save, "instance": arrlib.MidiArray()}

In [7]:
def change_res_fil(midifile,n_res):
        o_res = midifile.ticks_per_beat
        
        if o_res == n_res:
            return midifile
        
        frac = n_res/o_res
        for t in midifile.tracks:
            for e in t:
                e.time = int(round(frac*e.time))
        midifile.ticks_per_beat = n_res

        return midifile

def midifile_save(src,path):
    src.save(path)
    return path

def midifile_load(path,res):
    midi_data = fillib.MidiFile(path)
    midi_data = change_res_fil(midi_data,res)
    return midi_data

midifile_repr = {"load": midifile_load, "save": midifile_save, "instance": fillib.MidiFile()}

In [8]:
path = "samples/"

_____________________________________________
#### Checking ticks per beat vs ticks per quarter

In [9]:
midifile = midifile_load(path+"3qs.mid",480) # -> File of a 3/4 measure with 3 quarters, three beats
midifile.ticks_per_beat
for track in midifile.tracks:
    tick = 0
    for event in track:
        event.time += tick
        tick = event.time
    print(tick/midifile.ticks_per_beat)

midifile1 = midifile_load(path+"3qc.mid",480) # -> File of a 6/8 measure with 3 quarters, two beats
midifile1.ticks_per_beat
for track in midifile1.tracks:
    tick = 0
    for event in track:
        event.time += tick
        tick = event.time
    print(tick/midifile1.ticks_per_beat)
    
print("Conclusion 'ticks_per_beats' is actually 'ticks_per_quarter_note'")

2.95
2.95
Conclusion 'ticks_per_beats' is actually 'ticks_per_quarter_note'


______________________________________
#### Checking relationship between ticks and time

In [10]:
#midifile.ticks2seconds()

_________________________________
## Loading

In [11]:
path1 = path + "orchestrate.mid"
pretty = pretty_load(path1,48)
pattern = pattern_load(path1,48)
multitrack = multitrack_load(path1,48)
midiarray = midiarray_load(path1,48)
midifile = midifile_load(path1,48)

##### Checking Some Parameter

In [12]:
pretty.resolution, pattern.resolution, multitrack.beat_resolution, midifile.ticks_per_beat, midiarray.res

(48, 48, 48, 48, 48)

In [13]:
#Intruments and tracks
len(pretty.instruments), len(multitrack.tracks), midiarray.array.shape[-1],  len(pattern), len(midifile.tracks)

(8, 8, 8, 9, 9)

## Checking Parameter Preservation for Saving and Loading

In [14]:
#Pretty save
pathpre = path+"preo.mid"
pretty_save(pretty,pathpre)
pre = pretty_load(pathpre,48)
pat = pattern_load(pathpre,48)
mul = multitrack_load(pathpre,48)
arr = midiarray_load(pathpre,48)
fil = midifile_load(pathpre,48)
print("Resolution:\t|Pretty:",pre.resolution, "\t|Pattern:", pat.resolution, "\t|Multitrack:", mul.beat_resolution, "\t|MidiFile:", fil.ticks_per_beat, "\t|MidiArray:", arr.res)
print("Tr/Instr N:\t|Pretty:",len(pre.instruments), "\t|Pattern:", len(pat), "\t|Multitrack:", len(mul.tracks), "\t\t|MidiFile:", len(fil.tracks), "\t|MidiArray:", arr.array.shape[-1])

Resolution:	|Pretty: 48 	|Pattern: 48 	|Multitrack: 48 	|MidiFile: 48 	|MidiArray: 48
Tr/Instr N:	|Pretty: 8 	|Pattern: 9 	|Multitrack: 8 		|MidiFile: 9 	|MidiArray: 8


In [15]:
#Pattern Save
pathpat = path+"pato.mid"
pattern_save(pattern,pathpat)
pre = pretty_load(pathpat,48)
pat = pattern_load(pathpat,48)
mul = multitrack_load(pathpat,48)
arr = midiarray_load(pathpat,48)
fil = midifile_load(pathpat,48)
print("Resolution:\t|Pretty:",pre.resolution, "\t|Pattern:", pat.resolution, "\t|Multitrack:", mul.beat_resolution, "\t|MidiFile:", fil.ticks_per_beat, "\t|MidiArray:", arr.res)
print("Tr/Instr N:\t|Pretty:",len(pre.instruments), "\t|Pattern:", len(pat), "\t|Multitrack:", len(mul.tracks), "\t\t|MidiFile:", len(fil.tracks), "\t|MidiArray:", arr.array.shape[-1])

Resolution:	|Pretty: 48 	|Pattern: 48 	|Multitrack: 48 	|MidiFile: 48 	|MidiArray: 48
Tr/Instr N:	|Pretty: 8 	|Pattern: 9 	|Multitrack: 8 		|MidiFile: 9 	|MidiArray: 8


In [16]:
#Multitrack Save
pathmul = path+"mulo.mid"
multitrack_save(multitrack,pathmul)
pre = pretty_load(pathmul,48)
pat = pattern_load(pathmul,48)
mul = multitrack_load(pathmul,48)
arr = midiarray_load(pathmul,48)
fil = midifile_load(pathmul,48)
print("Resolution:\t|Pretty:",pre.resolution, "\t|Pattern:", pat.resolution, "\t|Multitrack:", mul.beat_resolution, "\t|MidiFile:", fil.ticks_per_beat, "\t|MidiArray:", arr.res)
print("Tr/Instr N:\t|Pretty:",len(pre.instruments), "\t|Pattern:", len(pat), "\t|Multitrack:", len(mul.tracks), "\t\t|MidiFile:", len(fil.tracks), "\t|MidiArray:", arr.array.shape[-1])

Resolution:	|Pretty: 48 	|Pattern: 48 	|Multitrack: 48 	|MidiFile: 48 	|MidiArray: 48
Tr/Instr N:	|Pretty: 8 	|Pattern: 9 	|Multitrack: 8 		|MidiFile: 9 	|MidiArray: 8


In [17]:
#Midfile Save
pathfil = path+"filo.mid"
midifile_save(midifile,pathfil)
pre = pretty_load(pathfil,48)
pat = pattern_load(pathfil,48)
mul = multitrack_load(pathfil,48)
arr = midiarray_load(pathfil,48)
fil = midifile_load(pathfil,48)
print("Resolution:\t|Pretty:",pre.resolution, "\t|Pattern:", pat.resolution, "\t|Multitrack:", mul.beat_resolution, "\t|MidiFile:", fil.ticks_per_beat, "\t|MidiArray:", arr.res)
print("Tr/Instr N:\t|Pretty:",len(pre.instruments), "\t|Pattern:", len(pat), "\t|Multitrack:", len(mul.tracks), "\t\t|MidiFile:", len(fil.tracks), "\t|MidiArray:", arr.array.shape[-1])

Resolution:	|Pretty: 48 	|Pattern: 48 	|Multitrack: 48 	|MidiFile: 48 	|MidiArray: 48
Tr/Instr N:	|Pretty: 8 	|Pattern: 9 	|Multitrack: 8 		|MidiFile: 9 	|MidiArray: 8


In [18]:
#Midi Array Save
patharr = path+"arro.mid"
midiarray_save(midiarray,patharr)
pre = pretty_load(patharr,48)
pat = pattern_load(patharr,48)
mul = multitrack_load(patharr,48)
arr = midiarray_load(patharr,48)
fil = midifile_load(patharr,48)
print("Resolution:\t|Pretty:",pre.resolution, "\t|Pattern:", pat.resolution, "\t|Multitrack:", mul.beat_resolution, "\t|MidiFile:", fil.ticks_per_beat, "\t|MidiArray:", arr.res)
print("Tr/Instr N:\t|Pretty:",len(pre.instruments), "\t|Pattern:", len(pat), "\t|Multitrack:", len(mul.tracks), "\t\t|MidiFile:", len(fil.tracks), "\t|MidiArray:", arr.array.shape[-1])

Resolution:	|Pretty: 48 	|Pattern: 48 	|Multitrack: 48 	|MidiFile: 48 	|MidiArray: 48
Tr/Instr N:	|Pretty: 8 	|Pattern: 9 	|Multitrack: 8 		|MidiFile: 9 	|MidiArray: 8
